# CORE Cartridge Notebook:: Cancel Before Active Enrichment
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do.

![caution](assets/cautionTape.png)

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

Cancelled/Discontinued Before Active enrichment.
Assigns hierarchy values in cases where cancelled or discontinued status is reported before first active shipment.  This is used as part of the TTFF enrichment.  See logic diagram below:

<img src='assets/cancel_before_active.svg' width=700>

In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


### Configuration

In [ ]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    input_transform: str = db_transform.variables.input_transform # Transform to source data from
    hierarchy: str = db_transform.variables.hierarchy # Column header for Patient Journey Hierarchy
    active_substatus_code: str = db_transform.variables.active_substatus_code # Active Shipment Substatus code, e.g. 'SHIPMENT' (customer-specific)
    cancel_discontinue_status_code: str = db_transform.variables.cancel_discontinue_status_code # Comma-separated list (stored as string) of Cancelled and Discontinued status codes, e.g. 'CANCELLED,DISCONTINUED' (customer-specific)
    cancel_discontinue_status_code = cancel_discontinue_status_code.split(',') # We reassign the string variable to be a list of strings by comma split
    bvpa_cancel_discontinue_substatus: str = db_transform.variables.bvpa_cancel_discontinue_substatus # Comma-separated list (stored as string) of accepted substatus codes used for BVPA hierarchy, e.g. 'INSURANCE DENIED,COVERAGE DENIED' (customer-specific)
    bvpa_cancel_discontinue_substatus = bvpa_cancel_discontinue_substatus.split(',') # We reassign the string variable to be a list of strings by comma split
    active_diff_threshold: int = db_transform.variables.active_diff_threshold # Threshold value for Active/Cancel date difference logic (customer-specific)
    prior_diff_threshold: int = db_transform.variables.prior_diff_threshold # Threshold value for Cancel/Prior date difference logic (customer-specific)
    active_hierarchy: str = db_transform.variables.active_hierarchy # Hierarchy to assign to statuses after the first fill, e.g. 'ACTIVE - SHIPMENT' (customer-specific)
    remove_from_ttff: str = db_transform.variables.remove_from_ttff # Hierarchy to assign to statuses that are ignored from TTFF (customer-specific)
    no_status_clarity: str = db_transform.variables.no_status_clarity # Hierarchy to assign to cancelled/discontinued statuses with no status clarity (customer-specific)
    bvpa_hierarchy: str = db_transform.variables.bvpa_hierarchy # Hierarchy to assign to cancelled/discontinued statuses that have BVPA substatus (customer-specific)

In [ ]:
transform = Transform()

### Transformation

In [ ]:
### Retrieve current dataset from contract
from core.dataset_diff import DatasetDiff

diff = DatasetDiff(db_transform.id)
df = diff.get_diff(transform_name=transform.input_transform, values=[run_id])

In [ ]:
df.shape

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [ ]:
# Column names defined here from the pre-defined patient status schema

brand_col = 'brand'
patient_id = 'longitudinal_patient_id'
pharmacy = 'pharmacy_name'
status_date = 'status_date'
referral_date = 'referral_date'
status =  'status'
substatus =  'substatus'
hierarchy = transform.hierarchy

if DbTransform.pharmaceutical_company.upper() == 'SUN':
    trans_id = 'pharmacy_transaction_id'

else:
    trans_id = 'aggregator_transaction_id'
    

### DATA CLEANING: ADDRESS THIS SECTION BEFORE PIPELINE INTEGRATION

### APPLY TRANSFORM LOGIC

In [ ]:
# Assign Patient Journey (pj_id) and Patient Journey Step (pj_step) identifiers
# (These IDs are used for calculation purposes only.  They will not be published)

def pj(df):
    pj_df = (
        df
        .assign(**{
            'pj_id' : lambda x: (
                x.groupby([patient_id, pharmacy, brand_col]).grouper.group_info[0]
            ),
            'pj_step' : lambda x: x.index
        })
        .sort_values(
            by=[patient_id, pharmacy, brand_col, status_date, status, trans_id],
            ascending=[True, True, True, True, False, True])
        .reset_index(drop=True)
    )
    return pj_df

In [ ]:
# Filter to only include patient journeys where:
#    a) Active Shipment status is reported
#    b) Cancelled or Discontinued status occurs prior to first active shipment

def cancel_before_active(pj_df):
    cancel_before_active_df = (
        pj_df
        .assign(active_step = lambda x: (
            np.where(
                x[substatus] == transform.active_substatus_code,
                x['pj_step'],
                np.nan
        )))
        .assign(active_status_date = lambda x: (
            pd.to_datetime(np.where(
                x[substatus] == transform.active_substatus_code,
                x[status_date],
                pd.NaT
        ))))
        .assign(first_active_step = lambda x: (
            x.groupby(['pj_id'])['active_step']
            .transform(min)
        ))
        .assign(first_active_status_date = lambda x: (
            x.groupby(['pj_id'])['active_status_date']
            .transform(min)
        ))
        .drop(['active_step', 'active_status_date'], axis=1)
        .assign(active_cancel_diff = lambda x:(
            np.where(x[status].isin(transform.cancel_discontinue_status_code),
                     (x['first_active_status_date'] - x[status_date]) / np.timedelta64(1, 'D'),
                     np.nan
                    )
        ))
        .assign(active_cancel_diff = lambda x: (
            x.groupby(['pj_id'], sort=False)['active_cancel_diff']
            .transform(lambda x: x.bfill())
            ))
        .loc[lambda x: (
            x['pj_id'].isin(x
                            .loc[x['active_cancel_diff'] >= 0]
                            .pj_id
                            .drop_duplicates()
                            .tolist()
                           )
        )]
    )
    return cancel_before_active_df

In [ ]:
# For each patient journey step, get the previous status. If it's the first step in the patient journey, show "no_prior_status"
# For cancelled or discontinued statuses, get the time spent in previous status (if >= 60 days) - and then backfill values for that patient journey.

def prior_status(cancel_before_active_df):
    prior_status_df = (
        cancel_before_active_df
        .assign(prior_status = lambda x:(
            x.groupby(['pj_id'])[status]
            .transform(lambda x: x.shift(1))
        ))
        .fillna(value={'prior_status':'no_prior_status'})
        .assign(prior_status_diff = lambda x: (
            np.where(
                (x[status].isin(transform.cancel_discontinue_status_code)) & ((x[status_date] - x[status_date].shift(1))/np.timedelta64(1,'D') >= 60),
                (x
                 .groupby(['pj_id'])[status_date]
                 .transform(lambda x: (x - x.shift(1))/np.timedelta64(1,'D'))),
                np.nan       
            )
        ))
        .assign(prior_status_diff = lambda x: (
            x.groupby(['pj_id'], sort=False)['prior_status_diff']
            .transform(lambda x: x.bfill())
        ))
    )
    return prior_status_df

In [ ]:
# Apply logic to determine patient journey hierarchy. See logic diagram in transform description.

def hierarchy(prior_status_df):
    hierarchy_df = (
        prior_status_df
        .assign(**{
            hierarchy : lambda x:(
                np.where(
                    x['pj_step'] >= x['first_active_step'],
                    transform.active_hierarchy,
                    np.where(
                        x['active_cancel_diff'] > transform.active_diff_threshold,
                        transform.remove_from_ttff,
                        np.where(
                            (~x[status].isin(transform.cancel_discontinue_status_code)),
                            np.where(
                                x['prior_status_diff'] > transform.prior_diff_threshold,
                                transform.remove_from_ttff,
                                x[hierarchy]
                            ),
                            np.where(
                                (x['prior_status_diff'] > transform.prior_diff_threshold) | (x['prior_status'] == 'no_prior_status'),
                                transform.no_status_clarity,
                                np.where(
                                    x[substatus].isin(transform.bvpa_cancel_discontinue_substatus),
                                    transform.bvpa_hierarchy,
                                    None
                                )
                            )
                        )
                    )
                )
            )
        })
                
        .reset_index(drop=True)
        .assign(**{
            hierarchy : lambda x: (
                x.groupby(['pj_id'], sort=False)[hierarchy]
                .transform(lambda x: x.ffill())
                )
        })
    )
    return hierarchy_df

In [ ]:
pj_df = pj(df)

pj_df.head()

In [ ]:
hierarchy_df = (
    pj_df
    .pipe(cancel_before_active)
    .pipe(prior_status)
    .pipe(hierarchy)
)

hierarchy_df.head()

In [ ]:
# Merge hierarchy results for this enrichment back into the initial dataframe

final_dataframe = (
    pd.merge(
        pj_df.rename(columns = {hierarchy:'old_hierarchy'}),
        hierarchy_df.loc[:,['pj_id', 'pj_step', hierarchy]],
        how='left',
        on=['pj_id', 'pj_step']
    )
    .assign(**{
        hierarchy : lambda x:(
            np.where(
                x[hierarchy].isnull(),
                x['old_hierarchy'],
                x[hierarchy]
            )
        )}
    )
    .drop(['pj_id', 'pj_step', 'old_hierarchy'], axis=1)
)

final_dataframe.head()

In [ ]:
final_dataframe.shape

### TEST TRANSFORM OUTPUT

In [ ]:
# TEST 1: Check that final dataframe has the same number of rows as the input dataframe

test1 = (pj_df.shape[0] == final_dataframe.shape[0])

test1

In [ ]:
# TEST 2: Check that hierarchy is not changed for statuses after first active shipment, or for journeys that never reported active shipment

first_active_status = (
    final_dataframe
    .assign(pj_id = lambda x: x.groupby([patient_id, pharmacy, brand_col]).grouper.group_info[0])
    .assign(pj_step = lambda x: x.index)
    .assign(active_status_date = lambda x: (
        pd.to_datetime(np.where(
            x[substatus] == transform.active_substatus_code,
            x[status_date],
            pd.NaT
    ))))
    .assign(first_active_status_date = lambda x: (
        x.groupby(['pj_id'])['active_status_date']
        .transform(min)
    ))
    .drop(['active_status_date'], axis=1)
    .merge(
        pj_df.loc[:,['pj_step', hierarchy]].rename(columns={hierarchy:'old_hierarchy'}),
        how = 'inner',
        on = ['pj_step']
    )
)

test2 = (first_active_status   
    .loc[lambda x: (
        ((x[status_date] > x['first_active_status_date'])
         |
         (x['first_active_status_date'].isnull())
        )
        &
        (x[hierarchy] != x['old_hierarchy'])
        &
        ~((x[hierarchy].isnull()) & (x['old_hierarchy'].isnull()))
        &
        (x[hierarchy] != transform.active_hierarchy)
    )]
)

test2 = (test2.shape[0] == 0)

test2

In [ ]:
# TEST 3: Check that all cancelled and discontinued statuses prior to first active shipment have a new hierarchy assigned to them

test3 = (
    first_active_status   
    .loc[lambda x: (
        (x[status_date] < x['first_active_status_date'])
        &
        (x[status].isin(transform.cancel_discontinue_status_code))
        &
        (
            ((x['first_active_status_date'] - x[status_date]) / np.timedelta64(1,'D') > transform.active_diff_threshold)
            |
            ((x[status_date] - x[status_date].shift(1)) / np.timedelta64(1,'D') > transform.prior_diff_threshold)
            |
            (x[substatus].isin(transform.bvpa_cancel_discontinue_substatus))
        )
        &
        (x[hierarchy] == x['old_hierarchy'])
    )]
)

test3 = (test3.shape[0] == 0)

test3

In [ ]:
# TEST 4: Check that all non-cancel/discontinue statuses prior to first active shipment have their previous hierarchy assignment (unless they are REMOVE FROM TTFF)

test4 = (
    first_active_status
    .loc[lambda x: (
        (x[status_date] < x['first_active_status_date'])
        &
        (x['pj_id'].isin(
            x
            .loc[(x[status].isin(transform.cancel_discontinue_status_code)) & (x[status_date] < x['first_active_status_date'])]
            .pj_id
            .drop_duplicates()
            .tolist()
        ))
        &
        (~x[status].isin(transform.cancel_discontinue_status_code))
        &
        (
            (x[hierarchy] != x['old_hierarchy'])
            &
            ~((x[hierarchy].isnull()) & (x['old_hierarchy'].isnull()))
            &
            (x[hierarchy] != transform.remove_from_ttff)
        )
    )]
)

test4 = (test4.shape[0] == 0)

test4

In [ ]:
# TEST 5: Create a "test" dataframe with expected results

test_data = ([
    [1, 0, 'PENDING', 'OTHER', 'PENDING - OTHER'],
    [1, 70, 'PENDING', 'OTHER', 'PENDING - OTHER'],
    [1, 72, transform.cancel_discontinue_status_code[0], transform.bvpa_cancel_discontinue_substatus[0], transform.bvpa_hierarchy],
    [1, 72, transform.cancel_discontinue_status_code[1], 'OTHER', transform.bvpa_hierarchy],
    [1, 72, 'ACTIVE', transform.active_substatus_code, transform.active_hierarchy],
    [2, 0, 'PENDING', 'OTHER', transform.remove_from_ttff],
    [2, 70, transform.cancel_discontinue_status_code[1], 'OTHER', transform.no_status_clarity],
    [2, 72, 'PENDING', 'OTHER', 'PENDING - OTHER'],
    [2, 72, transform.cancel_discontinue_status_code[0], 'OTHER', 'PENDING - OTHER'],
    [2, 72, 'ACTIVE', transform.active_substatus_code, transform.active_hierarchy],
    [3, 0, transform.cancel_discontinue_status_code[1], transform.bvpa_cancel_discontinue_substatus[0], transform.remove_from_ttff],
    [3, 1, 'PENDING', 'OTHER', transform.remove_from_ttff],
    [3, 2, transform.cancel_discontinue_status_code[1], 'OTHER', transform.remove_from_ttff],
    [3, 70, transform.cancel_discontinue_status_code[1], 'OTHER', transform.no_status_clarity],
    [3, 71, 'ACTIVE', transform.active_substatus_code, transform.active_hierarchy],
    [3, 72, transform.cancel_discontinue_status_code[0], 'OTHER', transform.active_hierarchy],
    [3, 73, transform.cancel_discontinue_status_code[0], transform.bvpa_cancel_discontinue_substatus[0], transform.active_hierarchy]
])

test_df = (
    pd.DataFrame(test_data, columns = [patient_id, status_date, status, substatus, 'expected_hierarchy'])
    .assign(**{
        pharmacy : 'ABC',
        brand_col : 'A',
        status_date : lambda x: (
            pd.to_datetime('2019-01-01', format='%Y-%m-%d') + pd.to_timedelta(x[status_date], unit='d')
        ),
        hierarchy : lambda x: (
            x[status] + ' - ' + x[substatus]
        )
    })
    
)

test_df

In [ ]:
# Apply transform to test dataframe

pj_test = pj(test_df)

test_output = (
    pj_test
    .pipe(cancel_before_active)
    .pipe(prior_status)
    .pipe(hierarchy)
)

final_dataframe_test = (
    pd.merge(
        pj_test.rename(columns = {hierarchy:'old_hierarchy'}),
        test_output.loc[:,['pj_id','pj_step', hierarchy]],
        how='left',
        on=['pj_id', 'pj_step']
    )
    .assign(**{
        hierarchy : lambda x:(
            np.where(
                x[hierarchy].isnull(),
                x['old_hierarchy'],
                x[hierarchy]
            )
        )}
    )
    .drop(['pj_id', 'pj_step', 'old_hierarchy'], axis=1)
)

final_dataframe_test

In [ ]:
# Check that results match expectations

test5 = (
    final_dataframe_test
    .assign(passfail = lambda x: np.where(
        (x[hierarchy] == x['expected_hierarchy']) | (x[hierarchy].isnull() & x['expected_hierarchy'].isnull()),
        True,
        False
    ))
    .passfail
    .all()
)

test5

In [ ]:
# FINAL TEST: Did all 5 tests pass?

test1 & test2 & test3 & test4 & test5

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session, publish_to_redshift=False) # Remove publish_to_redshift=False before pipeline integration!
session.close()